In [1]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MaxAbsScaler
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

In [3]:
url_file = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat'
columns = ['frequency',
           'angle_of_attack', 
           'chord_length', 
           'free_stream_velocity',
           'suction_side_displacement_thickness',
           'scaled_sound_pressure_level']

features = ['frequency','angle_of_attack', 
            'chord_length', 
            'free_stream_velocity',
            'suction_side_displacement_thickness']

df = pd.read_csv(url_file, sep='\t', header=None, names=columns)

In [4]:
df.head()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [5]:
df.tail()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224
1502,6300,15.6,0.1016,39.6,0.052849,104.204


In [6]:
df.describe()

,frequency,angle_of_attack,chord_length,free_stream_velocity,suction_side_displacement_thickness,scaled_sound_pressure_level
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000
mean,2886.380572,6.782302,0.136548,50.860745,0.011140,124.835943
std,3152.573137,5.918128,0.093541,15.572784,0.013150,6.898657
min,200.000000,0.000000,0.025400,31.700000,0.000401,103.380000
25%,800.000000,2.000000,0.050800,39.600000,0.002535,120.191000
50%,1600.000000,5.400000,0.101600,39.600000,0.004957,125.721000
75%,4000.000000,9.900000,0.228600,71.300000,0.015576,129.995500
max,20000.000000,22.200000,0.304800,71.300000,0.058411,140.987000


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503 entries, 0 to 1502
Data columns (total 6 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   frequency                            1503 non-null   int64  
 1   angle_of_attack                      1503 non-null   float64
 2   chord_length                         1503 non-null   float64
 3   free_stream_velocity                 1503 non-null   float64
 4   suction_side_displacement_thickness  1503 non-null   float64
 5   scaled_sound_pressure_level          1503 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 70.6 KB


In [8]:
def preprocess_input(df):
    
    train_data = df.copy()
    train_set = train_data.drop("scaled_sound_pressure_level", axis = 1)
    train_labels = train_data["scaled_sound_pressure_level"].copy()
    
    train_set = QuantileTransformer(random_state=0).fit_transform(train_set)
    train_set = MaxAbsScaler().fit_transform(train_set)
    train_set = StandardScaler().fit_transform(train_set)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(train_set, train_labels, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = preprocess_input(df)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1202, 5)
(301, 5)
(1202,)
(301,)


In [11]:
print(type(X_train), type(y_train))

<class 'numpy.ndarray'> <class 'pandas.core.series.Series'>


In [12]:
X_train

array([[ 1.66117102, -1.4811186 , -0.70593673, -1.45688844, -1.52418124],
       [-0.54503446,  0.21672565, -0.1884601 ,  1.29034346, -0.05031339],
       [-1.73045831,  1.34810308, -0.1884601 ,  1.29034346,  1.60540655],
       ...,
       [ 0.87262154, -0.23613656,  0.91797644,  0.18760073, -0.27892744],
       [ 1.47399884, -1.4811186 ,  1.58152757,  1.29034346, -0.82794754],
       [-1.48956077,  1.07887571,  0.36475817,  1.29034346,  1.63831311]])

In [13]:
y_train

776     119.070
1346    136.758
1473    130.898
1030    116.366
1191    124.154
         ...   
141     110.652
1430    124.244
326     120.654
10      117.151
695     128.484
Name: scaled_sound_pressure_level, Length: 1202, dtype: float64

In [11]:
model = Sequential()

model.add(Input(X_train.shape[1],))

model.add(Dense(48, activation = 'relu'))

model.add(Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 49        
                                                                 
Total params: 337 (1.32 KB)
Trainable params: 337 (1.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.compile(loss = 'mean_squared_error', metrics = tf.keras.metrics.RootMeanSquaredError())

In [13]:
def get_shape(model):
    weights_layer = model.get_weights()
    shapes = []
    for weights in weights_layer:
        shapes.append(weights.shape)
    return shapes

In [14]:
def set_shape(weights,shapes):
    new_weights = []
    index=0
    for shape in shapes:
        if(len(shape)>1):
            n_nodes = np.prod(shape) + index
        else:
            n_nodes=shape[0]+index
        tmp = np.array(weights[index:n_nodes]).reshape(shape)
        new_weights.append(tmp)
        index=n_nodes
    return new_weights

In [15]:
shape = get_shape(model)
num_solutions = 15
dimensions = 337

In [16]:
def evaluate_nn(W):
    model.set_weights(set_shape(W,shape))
    score = model.evaluate(X_train, y_train, verbose = 0)
    return score[1]

In [17]:
# generate the solutions

current_population_vectors = np.random.uniform(size = (num_solutions, dimensions))

print(current_population_vectors.shape)

(15, 337)


In [18]:
epochs, global_best_error, best_solution = 1000, float("inf"), None

In [19]:
import Grey_Wolves as GWO

In [20]:
for epoch in range(epochs):
    
    print("Epoch ", epoch + 1)
    
    # calculate error of each solution
    curr_errors = np.empty(shape=(num_solutions))
    for i in range(num_solutions):
        curr_errors[i] = evaluate_nn(current_population_vectors[i])
        
    # get the current best error
    current_best_error, idx = np.min(curr_errors), np.argmin(curr_errors)
    if(current_best_error < global_best_error):
        global_best_error = current_best_error
        best_solution = current_population_vectors[idx]
        
    print("Current best: ", current_best_error, " Global best: ", global_best_error)
    
    # get the alpha, beta and delta of the population
    X_alpha, X_beta, X_delta = GWO.find_alpha_beta_delta(current_population_vectors, curr_errors)
    x_alpha_score = evaluate_nn(X_alpha)
    x_beta_score = evaluate_nn(X_beta)
    x_delta_score = evaluate_nn(X_delta)
    print("Top three least errors : ", x_alpha_score, x_beta_score, x_delta_score)
    
    # Generate a_alpha, a_beta, a_delta

    a_alpha = 1 / np.cosh(4 * epoch/epochs)
    a_delta = 1 / np.cosh(3 * epoch/epochs)
    a_beta = (a_alpha + a_delta) * 0.5
    
    # Update the new population
    new_population_vectors = GWO.HGWO(current_population_vectors, X_alpha, X_beta, X_delta, a_alpha, a_beta, a_delta)
    new_errors = np.empty(shape=(num_solutions))
    for i in range(num_solutions):
        new_errors[i] = evaluate_nn(new_population_vectors[i])
    
    # compare the two populations and make changes accordingly
    idxs = np.where(new_errors < curr_errors)
    print(idxs)
    current_population_vectors[idxs] = new_population_vectors[idxs]
    print()

Epoch  1
Current best:  104.42564392089844  Global best:  104.42564392089844
Top three least errors :  104.42564392089844 104.42730712890625 105.11310577392578
(array([ 0,  1,  2,  3,  5,  6,  7,  8, 10, 11, 12, 13, 14], dtype=int64),)

Epoch  2
Current best:  103.02629852294922  Global best:  103.02629852294922
Top three least errors :  103.02629852294922 103.57318115234375 104.03761291503906
(array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13], dtype=int64),)

Epoch  3
Current best:  102.3820571899414  Global best:  102.3820571899414
Top three least errors :  102.3820571899414 102.74258422851562 102.94629669189453
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64),)

Epoch  4
Current best:  100.74812316894531  Global best:  100.74812316894531
Top three least errors :  100.74812316894531 101.2394027709961 101.5038070678711
(array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14], dtype=int64),)

Epoch  5
Current best:  99.2431869506836  Globa

(array([ 3,  6,  7,  8, 14], dtype=int64),)

Epoch  39
Current best:  28.307279586791992  Global best:  28.307279586791992
Top three least errors :  28.307279586791992 28.742265701293945 28.838626861572266
(array([ 3,  5,  7,  8, 10, 12], dtype=int64),)

Epoch  40
Current best:  28.19620704650879  Global best:  28.19620704650879
Top three least errors :  28.19620704650879 28.307279586791992 28.445621490478516
(array([ 4, 10, 11], dtype=int64),)

Epoch  41
Current best:  25.963953018188477  Global best:  25.963953018188477
Top three least errors :  25.963953018188477 27.703853607177734 28.19620704650879
(array([ 2,  7, 12, 14], dtype=int64),)

Epoch  42
Current best:  25.963953018188477  Global best:  25.963953018188477
Top three least errors :  25.963953018188477 26.725215911865234 27.148948669433594
(array([1, 2, 3, 5, 6, 9], dtype=int64),)

Epoch  43
Current best:  25.963953018188477  Global best:  25.963953018188477
Top three least errors :  25.963953018188477 26.19092559814453 26.3

Current best:  14.543478965759277  Global best:  14.543478965759277
Top three least errors :  14.543478965759277 14.84812068939209 14.96155071258545
(array([ 1, 14], dtype=int64),)

Epoch  82
Current best:  14.300710678100586  Global best:  14.300710678100586
Top three least errors :  14.300710678100586 14.84812068939209 14.96155071258545
(array([ 5, 13, 14], dtype=int64),)

Epoch  83
Current best:  14.300710678100586  Global best:  14.300710678100586
Top three least errors :  14.300710678100586 14.77714729309082 14.84812068939209
(array([4], dtype=int64),)

Epoch  84
Current best:  14.300710678100586  Global best:  14.300710678100586
Top three least errors :  14.300710678100586 14.3641996383667 14.77714729309082
(array([10, 11], dtype=int64),)

Epoch  85
Current best:  14.300710678100586  Global best:  14.300710678100586
Top three least errors :  14.300710678100586 14.3641996383667 14.447864532470703
(array([ 2,  8, 12, 13], dtype=int64),)

Epoch  86
Current best:  13.917107582092285 

(array([ 1,  9, 13], dtype=int64),)

Epoch  124
Current best:  10.85836124420166  Global best:  10.85836124420166
Top three least errors :  10.85836124420166 11.271960258483887 11.378307342529297
(array([ 9, 10], dtype=int64),)

Epoch  125
Current best:  10.85836124420166  Global best:  10.85836124420166
Top three least errors :  10.85836124420166 11.223941802978516 11.271960258483887
(array([ 0,  2, 12, 14], dtype=int64),)

Epoch  126
Current best:  10.54400634765625  Global best:  10.54400634765625
Top three least errors :  10.54400634765625 10.703597068786621 10.85836124420166
(array([13, 14], dtype=int64),)

Epoch  127
Current best:  10.54400634765625  Global best:  10.54400634765625
Top three least errors :  10.54400634765625 10.703597068786621 10.757939338684082
(array([8], dtype=int64),)

Epoch  128
Current best:  10.54400634765625  Global best:  10.54400634765625
Top three least errors :  10.54400634765625 10.593052864074707 10.703597068786621
(array([], dtype=int64),)

Epoch  

Top three least errors :  9.21065616607666 9.568735122680664 9.570992469787598
(array([], dtype=int64),)

Epoch  168
Current best:  9.21065616607666  Global best:  9.21065616607666
Top three least errors :  9.21065616607666 9.568735122680664 9.570992469787598
(array([0], dtype=int64),)

Epoch  169
Current best:  9.198111534118652  Global best:  9.198111534118652
Top three least errors :  9.198111534118652 9.21065616607666 9.568735122680664
(array([12, 14], dtype=int64),)

Epoch  170
Current best:  9.198111534118652  Global best:  9.198111534118652
Top three least errors :  9.198111534118652 9.21065616607666 9.461806297302246
(array([6, 7], dtype=int64),)

Epoch  171
Current best:  9.198111534118652  Global best:  9.198111534118652
Top three least errors :  9.198111534118652 9.20660400390625 9.21065616607666
(array([ 1,  2, 10], dtype=int64),)

Epoch  172
Current best:  9.198111534118652  Global best:  9.198111534118652
Top three least errors :  9.198111534118652 9.20660400390625 9.2106

...

Current best:  5.34581995010376  Global best:  5.34581995010376
Top three least errors :  5.34581995010376 5.358916759490967 5.3728814125061035
(array([ 2, 10, 13], dtype=int64),)

Epoch  788
Current best:  5.34581995010376  Global best:  5.34581995010376
Top three least errors :  5.34581995010376 5.352417945861816 5.358916759490967
(array([12, 14], dtype=int64),)

Epoch  789
Current best:  5.337488174438477  Global best:  5.337488174438477
Top three least errors :  5.337488174438477 5.34581995010376 5.352417945861816
(array([], dtype=int64),)

Epoch  790
Current best:  5.337488174438477  Global best:  5.337488174438477
Top three least errors :  5.337488174438477 5.34581995010376 5.352417945861816
(array([ 1, 10], dtype=int64),)

Epoch  791
Current best:  5.334627151489258  Global best:  5.334627151489258
Top three least errors :  5.334627151489258 5.337488174438477 5.34581995010376
(array([ 1,  2, 13], dtype=int64),)

Epoch  792
Current best:  5.30237340927124  Global best:  5.3023734

(array([ 3, 11], dtype=int64),)

Epoch  831
Current best:  5.060054302215576  Global best:  5.060054302215576
Top three least errors :  5.060054302215576 5.078054904937744 5.089122772216797
(array([ 2,  3, 12], dtype=int64),)

Epoch  832
Current best:  5.049815654754639  Global best:  5.049815654754639
Top three least errors :  5.049815654754639 5.060054302215576 5.08282470703125
(array([4], dtype=int64),)

Epoch  833
Current best:  5.049815654754639  Global best:  5.049815654754639
Top three least errors :  5.049815654754639 5.053840637207031 5.060054302215576
(array([ 1, 12], dtype=int64),)

Epoch  834
Current best:  5.032347202301025  Global best:  5.032347202301025
Top three least errors :  5.032347202301025 5.049815654754639 5.053840637207031
(array([10], dtype=int64),)

Epoch  835
Current best:  5.032347202301025  Global best:  5.032347202301025
Top three least errors :  5.032347202301025 5.049815654754639 5.053840637207031
(array([10, 12, 13], dtype=int64),)

Epoch  836
Current 

Top three least errors :  4.747969627380371 4.756168365478516 4.761724472045898
(array([], dtype=int64),)

Epoch  875
Current best:  4.747969627380371  Global best:  4.747969627380371
Top three least errors :  4.747969627380371 4.756168365478516 4.761724472045898
(array([10], dtype=int64),)

Epoch  876
Current best:  4.747969627380371  Global best:  4.747969627380371
Top three least errors :  4.747969627380371 4.756168365478516 4.761724472045898
(array([2, 6], dtype=int64),)

Epoch  877
Current best:  4.747969627380371  Global best:  4.747969627380371
Top three least errors :  4.747969627380371 4.756168365478516 4.761724472045898
(array([], dtype=int64),)

Epoch  878
Current best:  4.747969627380371  Global best:  4.747969627380371
Top three least errors :  4.747969627380371 4.756168365478516 4.761724472045898
(array([ 7, 11], dtype=int64),)

Epoch  879
Current best:  4.747969627380371  Global best:  4.747969627380371
Top three least errors :  4.747969627380371 4.756168365478516 4.7617

(array([1, 2, 7, 8], dtype=int64),)

Epoch  918
Current best:  4.457261085510254  Global best:  4.457261085510254
Top three least errors :  4.457261085510254 4.4616193771362305 4.472482681274414
(array([6], dtype=int64),)

Epoch  919
Current best:  4.420584678649902  Global best:  4.420584678649902
Top three least errors :  4.420584678649902 4.457261085510254 4.4616193771362305
(array([], dtype=int64),)

Epoch  920
Current best:  4.420584678649902  Global best:  4.420584678649902
Top three least errors :  4.420584678649902 4.457261085510254 4.4616193771362305
(array([10, 14], dtype=int64),)

Epoch  921
Current best:  4.420584678649902  Global best:  4.420584678649902
Top three least errors :  4.420584678649902 4.457261085510254 4.4616193771362305
(array([ 9, 12], dtype=int64),)

Epoch  922
Current best:  4.41747522354126  Global best:  4.41747522354126
Top three least errors :  4.41747522354126 4.420584678649902 4.457261085510254
(array([ 1, 11], dtype=int64),)

Epoch  923
Current best

Current best:  4.263990879058838  Global best:  4.263990879058838
Top three least errors :  4.263990879058838 4.272582054138184 4.2774128913879395
(array([], dtype=int64),)

Epoch  962
Current best:  4.263990879058838  Global best:  4.263990879058838
Top three least errors :  4.263990879058838 4.272582054138184 4.2774128913879395
(array([ 7, 13], dtype=int64),)

Epoch  963
Current best:  4.263990879058838  Global best:  4.263990879058838
Top three least errors :  4.263990879058838 4.272582054138184 4.2774128913879395
(array([ 2, 13], dtype=int64),)

Epoch  964
Current best:  4.261508464813232  Global best:  4.261508464813232
Top three least errors :  4.261508464813232 4.263990879058838 4.271429061889648
(array([3, 6, 8], dtype=int64),)

Epoch  965
Current best:  4.261508464813232  Global best:  4.261508464813232
Top three least errors :  4.261508464813232 4.263990879058838 4.2670745849609375
(array([0], dtype=int64),)

Epoch  966
Current best:  4.260249614715576  Global best:  4.260249

In [21]:
model.set_weights(set_shape(best_solution, shape))

In [22]:
# RMSE
score = model.evaluate(X_train, y_train)
train_rmse, train_mse = score[1], score[0]

score = model.evaluate(X_test, y_test)
test_rmse, test_mse = score[1], score[0]

print("Train RMSE = ", train_rmse)
print("Test RMSE = ", test_rmse)
print("---------------------------")

#MSE
print("Train MSE = ", train_mse)
print("Test MSE = ", test_mse)
print("---------------------------")

#R2 Score
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_r2_score = r2_score(y_train, y_train_pred)
test_r2_score = r2_score(y_test, y_test_pred)

print("Train R2 = ", train_r2_score)
print("Test R2 = ", test_r2_score)
print("-----------------------------")

# MAE
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
print("Train MAE = ", train_mae)
print("Test MAE = ", test_mae)

10/10 [==============================] - 0s 3ms/step - loss: 17.2641 - root_mean_squared_error: 4.1550
Train RMSE =  4.202889919281006
Test RMSE =  4.155012130737305
---------------------------
Train MSE =  17.66428565979004
Test MSE =  17.264127731323242
---------------------------
10/10 [==============================] - 0s 2ms/step
Train R2 =  0.6194557402700156
Test R2 =  0.6681952485787721
-----------------------------
Train MAE =  3.1572003711059367
Test MAE =  3.1506153576619607
